In [1]:
import numpy as np
import random
import math

In [2]:
class Bayesian_Network:
  def __init__(self, bn, cpm, initial_samples):
    self.cps = cpm
    self.initial_samples = initial_samples
    self.bayesian_net = bn
  
  def get_parents(self):
    bayesian_net = self.bayesian_net
    parents = {}

    for var in list(bayesian_net.keys()):
      parents[var] = []
    variables = list(bayesian_net.keys())

    for var in variables:
      for node in bayesian_net:
        if var in bayesian_net[node]:
          parents[var].append(node)

    return parents

  def get_evidence(self):
    samples, bayesian_net = self.initial_samples, self.bayesian_net
    evidence = {}
    for var in list(bayesian_net.keys()):
      evidence[var] = []
    
    for var in list(bayesian_net.keys()):
      last_sample = samples[var][-1]
      evidence[var] = last_sample
    return evidence

  def p_a_given_parents(self, a, evidence):
    cpm_dict = self.cps
    parents_dict = self.get_parents()
    parents = parents_dict[a]
    cpm = cpm_dict[a]

    num_parents = len(parents)

    parent_evidence = []
    for parent in parents:
      parent_evidence.append(evidence[parent])
    
    # print("num parents", num_parents, parents)

    cpm_new = cpm.copy()
    if num_parents == 0:
      # print("inside if")
      if evidence[a] == 1:
        return cpm_new[0]
      else:
        return 1 - cpm_new[0]

    for ev in parent_evidence:
      # print("inside", ev)
      prob_a_given_parents = cpm_new[ev]
      # print(ev, prob_a_given_parents)
      cpm_new = prob_a_given_parents

    if evidence[a] == 1:
      return prob_a_given_parents
    else:
      return 1 - prob_a_given_parents

  def __do_bayesian_sampling(self, p):
    import random
    r = random.random()
    if r < p:
      return 1
    else:
      return 0

  def __gibbs_helper(self, flag, evidence, var, cps, parents, bayesian_net):
    evidence[var] = flag 
    prob = 1
    p_var_given_parents = self.p_a_given_parents(var, evidence)
    # print(p_var_given_parents)
    prob = prob*p_var_given_parents

    for child in bayesian_net[var]:
      p_child_given_its_parents = self.p_a_given_parents(child, evidence)
      # print(p_child_given_its_parents)
      prob = prob*p_child_given_its_parents

    return prob

  def gibbs_sampling(self, num_iter):

    import copy 
    # samples = copy.deepcopy(samples_out)
    samples = copy.deepcopy(self.initial_samples)
    bayesian_net = self.bayesian_net

    evidence = self.get_evidence()
    parents = self.get_parents()
    variables = list(bayesian_net.keys())

    for i in range(num_iter):
      for var in variables:
        p_star = [1, 1]

        flag = 0
        p_star[0] = self.__gibbs_helper(flag, evidence, var, cps, parents, bayesian_net)

        flag = 1
        p_star[1] = self.__gibbs_helper(flag, evidence, var, cps, parents, bayesian_net)

        p_var = p_star[1]/sum(p_star)
        new_value_of_var = self.__do_bayesian_sampling(p_var)

        samples[var].append(new_value_of_var)
    return samples

In [3]:
bayesian_net = {'burglary':['alarm'], 
                'earthquake': ['alarm'], 
                'alarm': ['john calls', 'mary calls'], 
                'john calls': [], 
                'mary calls':[]}
print(bayesian_net)

{'burglary': ['alarm'], 'earthquake': ['alarm'], 'alarm': ['john calls', 'mary calls'], 'john calls': [], 'mary calls': []}


In [28]:
samples_out = {'burglary':[1],             
                'earthquake': [1], 
                'alarm': [0], 
                'john calls': [1], 
                'mary calls':[0]}

In [36]:
cps = {'burglary': [0.01], 
        'earthquake': [0.02],
       'alarm': [[0.001, 0.29],
                 [0.94, 0.95]],
       'john calls':[0.9, 0.05],
        'mary calls':[0.7, 0.05]}

In [37]:
net = Bayesian_Network(bayesian_net, cps, samples_out)

In [38]:
samples = net.gibbs_sampling(20000)

In [39]:
earth = samples['burglary']

In [40]:
count = 0
for i in earth[10000:]:
  if i == 1:
    count = count + 1
print(count/10000)

0.0334
